### Status
- no existing library fulfills all requirements 
- core problem: model firmly baked into code everywhere in all of them
- no integrated runable system ready at the moment
- no user interface
- only able to analyze files
- core necessary parts:
  - abstractions for recording, preprocessor, model 
  - model, preprocessing disentangled from rest => allows runtime replacement of model
  
- `birdnetlib` provides at the very least starting points for continously running service via `arecord` on linux and for the missint nuts and bolts 

### Code example

In [1]:
import iSparrow.sparrow_model_base as spm
import iSparrow.sparrow_recording as spr
import iSparrow.preprocessor_base as spb
import iSparrow.utils as utils

import install_sparrow as sp

2024-03-21 23:00:34.030701: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from pathlib import Path
import pandas as pd

In [3]:
# make a mock install of sparrow
sp.install()

Creating iSparrow folders and downloading data... 
...reading config from /Users/haraldmack/Development/iSparrow/config/install_cfg.yml
...Making direcotries...
/Users/haraldmack/iSparrow /Users/haraldmack/iSparrow/models /Users/haraldmack/iSparrow_data /Users/haraldmack/iSparrow_output /Users/haraldmack/iSparrow/example
... Downloading model files...
... Downloading example files...
Installation finished


In [4]:
# variables for analysis
sigmoid_sensitivity = 1.0
num_threads = 12
min_conf = 0.25

# variables for recording
recording_path = sp.EXAMPLES / "soundscape.wav"
sample_rate = 48000
overlap = 0.0
sample_secs = 3.0
resample_type = "kaiser_fast"

#### steps
- read module defined as input 
- build preprocessor
- build model 
- build recording instance from them 
- analyze

In [5]:
ppd = utils.load_module("ppm", sp.MODELS / Path("birdnet_default") / "preprocessor.py")

In [6]:
md = utils.load_module("md", sp.MODELS / Path("birdnet_default") / "model.py")

In [7]:
preprocessor = ppd.Preprocessor(sample_rate=sample_rate, overlap=overlap, sample_secs=sample_secs, resample_type=resample_type)

model = md.Model(sp.MODELS / "birdnet_default", num_threads=5, sigmoid_sensitivity=1.)

Labels loaded.


In [8]:
recording = spr.SparrowRecording(preprocessor, model, sp.EXAMPLES / "soundscape.wav", min_conf=min_conf)

In [9]:
recording.analyze()

read audio data of duration 120.0 from /Users/haraldmack/iSparrow/example/soundscape.wav with sampling rate 48000
process audio data custom
process audio data custom: complete, read  40 chunks.
name of model:  birdnet_default_model


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [10]:
pd.DataFrame(recording.detections)

,start,end,label,confidence
0,0.0,3.0,Poecile atricapillus_Black-capped Chickadee,0.814056
1,3.0,6.0,Poecile atricapillus_Black-capped Chickadee,0.308286
2,9.0,12.0,Haemorhous mexicanus_House Finch,0.639378
3,18.0,21.0,Cyanocitta cristata_Blue Jay,0.435271
4,18.0,21.0,Clamator coromandus_Chestnut-winged Cuckoo,0.322587
5,21.0,24.0,Cyanocitta cristata_Blue Jay,0.329086
6,27.0,30.0,Falco columbarius_Merlin,0.265653
7,33.0,36.0,Junco hyemalis_Dark-eyed Junco,0.459063
8,36.0,39.0,Junco hyemalis_Dark-eyed Junco,0.353685
9,39.0,42.0,Haemorhous mexicanus_House Finch,0.253251


#### Load a different model

In [11]:
ppc = utils.load_module("ppm", sp.MODELS / Path("birdnet_custom") / "preprocessor.py")
mc = utils.load_module("md", sp.MODELS / Path("birdnet_custom") / "model.py")

In [12]:
preprocessor = ppc.Preprocessor(sample_rate=sample_rate, overlap=overlap, sample_secs=sample_secs, resample_type=resample_type)

model = mc.Model(default_model_path=sp.MODELS / "birdnet_default", model_path=sp.MODELS / "birdnet_custom", num_threads=5, sigmoid_sensitivity=1.)

Default classifier loaded
Custom classifier loaded
Labels loaded.


In [13]:
recording = spr.SparrowRecording(preprocessor, model, sp.EXAMPLES / "soundscape.wav", min_conf=min_conf)

In [14]:
recording.analyze()

read audio data of duration 120.0 from /Users/haraldmack/iSparrow/example/soundscape.wav with sampling rate 48000
process audio data custom
process audio data custom: complete, read  40 chunks.
name of model:  birdnet_custom_model


In [15]:
pd.DataFrame(recording.detections)

,start,end,label,confidence
0,0.0,3.0,Poecile atricapillus_Black-capped Chickadee,0.956534
1,3.0,6.0,Poecile atricapillus_Black-capped Chickadee,0.655960
2,6.0,9.0,Junco hyemalis_Dark-eyed Junco,0.383827
3,15.0,18.0,Cyanocitta cristata_Blue Jay,0.510273
4,18.0,21.0,Cyanocitta cristata_Blue Jay,0.980540
5,21.0,24.0,Cyanocitta cristata_Blue Jay,0.979032
6,24.0,27.0,Cyanocitta cristata_Blue Jay,0.598468
7,27.0,30.0,Cyanocitta cristata_Blue Jay,0.496739
8,27.0,30.0,Junco hyemalis_Dark-eyed Junco,0.327117
9,30.0,33.0,Junco hyemalis_Dark-eyed Junco,0.310007


In [16]:
ppc = utils.load_module("ppm", sp.MODELS / Path("google_perch") / "preprocessor.py")
mc = utils.load_module("md", sp.MODELS / Path("google_perch") / "model.py")

In [17]:
preprocessor = ppc.Preprocessor(sample_rate=32000, sample_secs=5., resample_type=resample_type)
model = mc.Model(model_path=sp.MODELS / "google_perch", num_threads=5)

Labels loaded.


In [18]:
recording = spr.SparrowRecording(preprocessor, model, sp.EXAMPLES / "soundscape.wav", min_conf=min_conf)

In [19]:
recording.analyze()

read audio data of duration 120.0 from /Users/haraldmack/iSparrow/example/soundscape.wav with sampling rate 32000
process audio data custom 
process audio data google: complete, read  24 chunks.
name of model:  google_perch_model


I0000 00:00:1711058449.011199 2417314 service.cc:145] XLA service 0x7fae2e8e1450 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1711058449.011249 2417314 service.cc:153]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1711058452.389379 2417314 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [20]:
pd.DataFrame(recording.detections)

,start,end,label,confidence
0,0.0,5.0,bkcchi,0.502205
1,5.0,10.0,sonthr1,0.817928
2,10.0,15.0,tawowl1,0.345437
3,40.0,45.0,gretit1,0.343007
4,65.0,70.0,tawowl1,0.321463
5,75.0,80.0,tawowl1,0.351758
6,85.0,90.0,tawowl1,0.307805
7,85.0,90.0,eurrob1,0.261202
8,90.0,95.0,tawowl1,0.387926
9,105.0,110.0,tawowl1,0.310031


In [22]:
ppc = utils.load_module("ppm", sp.MODELS / Path("birdnet_default") / "preprocessor.py")
mc = utils.load_module("md", sp.MODELS / Path("birdnet_default") / "model.py")

preprocessor = ppd.Preprocessor(sample_rate=sample_rate, overlap=overlap, sample_secs=sample_secs, resample_type=resample_type)

model = md.Model(sp.MODELS / "birdnet_default", num_threads=5, sigmoid_sensitivity=1.)

recording.set_model_system(model, preprocessor)

print(recording.analyzer.name)
print(recording.processor.name, recording.processor.sample_rate)

Labels loaded.
birdnet_default_model
birdnet_defaults_preprocessor 48000


In [23]:
recording.analyze()

read audio data of duration 120.0 from /Users/haraldmack/iSparrow/example/soundscape.wav with sampling rate 48000
process audio data custom
process audio data custom: complete, read  40 chunks.
name of model:  birdnet_default_model


In [24]:
print(pd.DataFrame(recording.detections))

    start    end                                        label  confidence
0     0.0    3.0  Poecile atricapillus_Black-capped Chickadee    0.814056
1     3.0    6.0  Poecile atricapillus_Black-capped Chickadee    0.308286
2     9.0   12.0             Haemorhous mexicanus_House Finch    0.639378
3    18.0   21.0                 Cyanocitta cristata_Blue Jay    0.435271
4    18.0   21.0   Clamator coromandus_Chestnut-winged Cuckoo    0.322587
5    21.0   24.0                 Cyanocitta cristata_Blue Jay    0.329086
6    27.0   30.0                     Falco columbarius_Merlin    0.265653
7    33.0   36.0               Junco hyemalis_Dark-eyed Junco    0.459063
8    36.0   39.0               Junco hyemalis_Dark-eyed Junco    0.353685
9    39.0   42.0             Haemorhous mexicanus_House Finch    0.253251
10   42.0   45.0               Junco hyemalis_Dark-eyed Junco    0.737543
11   51.0   54.0             Haemorhous mexicanus_House Finch    0.256404
12   54.0   57.0             Haemorhou